In [2]:
%pip -q install google-genai

In [3]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [4]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [8]:
# Instalar Framework ADK de agentes do Google ################################################
!pip -q install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [9]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [10]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [11]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [28]:
##########################################
# --- Agente 1: Redator de histórias --- #
##########################################

def agente_redator(topico, nome_personagem_principal):
  redator = Agent(
      name = "agente_redator",
      model = "gemini-2.0-flash",
      description = "Agente que escreve histórias",
      tools = [google_search],
      instruction = """
      Você é um criador de histórias especializado em construir narrativas originais e envolventes.
      Receberá um tema e o nome do personagem principal.
      Sua tarefa é criar uma história completa, com início, desenvolvimento e fim, que possa futuramente ser convertida em um jogo no estilo “Escolha sua Aventura”.
      A história deve conter:
      Um protagonista bem definido (nome já fornecido)
      Uma ambientação clara e criativa
      Elementos de tensão, conflito e progressão narrativa
      Dilemas morais ou desafios com múltiplas possibilidades de desfecho
      Um enredo com acontecimentos significativos que permitam uma posterior ramificação em decisões
      3 possíveis finais diferentes (ex: um bom, um ruim, um neutro), com caminhos lógicos que conduzem a cada um deles
      Escreva como se fosse um conto, com narrativa linear
      Pesquise no Google (google_search) histórias com esse tema para ajudar a servir de base.
      """
  )
  entrada_do_agente_redator = f"Tema: {tema}\nNome do personagem principal: {nome_personagem_principal}"
  roteiro = call_agent(redator, entrada_do_agente_redator)
  return roteiro

In [29]:
################################################
# --- Agente 2: Conversor  --- #
################################################
def agente_conversor(tema, roteiro_inicial):
    conversor = Agent(
        name="agente_conversor",
        model="gemini-2.0-flash",
        instruction="""
        Você é um designer de narrativas interativas e especialista em transformar histórias tradicionais em jogos do tipo "Escolha sua Aventura".
        Receberá como entrada uma história linear completa, com início, meio e fim, incluindo múltiplos finais possíveis.
        Sua tarefa é converter essa história em um roteiro estruturado de jogo interativo, onde o jogador poderá tomar decisões que afetam o rumo da narrativa.
        Para isso, siga estas instruções:
        1. Divida a história em cenas interativas numeradas (por exemplo: Cena 1, Cena 2...)
        Cada cena deve ter:
        Uma descrição imersiva do momento atual
        De 2 a 4 opções de escolha para o jogador
        Para cada escolha, defina:
        Um texto da escolha (o que o jogador está fazendo)
        A consequência imediata (em uma linha)
        Para onde essa escolha leva (ex: Cena 2A, Cena 3B etc.)
        2. Crie caminhos ramificados:
        Permita que algumas escolhas levem a caminhos alternativos com eventos exclusivos.
        Inclua oportunidades de ganhar itens, aliados ou informações.
        Algumas escolhas podem encerrar o jogo (finais bons, ruins ou neutros).
        """,
        description="Agente que converte histórias no estilo Escolha sua Aventura",
        tools=[google_search]
    )

    entrada_do_agente_conversor = f"Tema:{tema}\nRoteiro inicial: {roteiro_inicial}"
    # Executa o agente
    roteiro_adaptado = call_agent(conversor, entrada_do_agente_conversor)
    return roteiro_adaptado

In [30]:
######################################
# --- Agente 3: Revisor --- #
######################################
def agente_revisor(tema, roteiro_adaptado):
    revisor = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um revisor de narrativa interativa com foco em tom, emoção e coerência lógica.
            Receberá um roteiro com cenas e decisões ramificadas para um jogo estilo "Escolha sua Aventura".
            Sua tarefa é revisar o conteúdo de forma completa:
            Verifique se cada decisão leva a uma consequência coerente e lógica.
            Enriqueça a narrativa com tom emocional adequado: suspense, humor, tensão, drama ou alívio.
            Torne as cenas mais imersivas com descrições vívidas, diálogos ou metáforas (sem exagero).
            Inclua elementos adicionais se fizerem sentido (ex: pistas ocultas, interações inesperadas, novos itens).
            """,
        description="Agente revisor"
    )
    entrada_do_agente_revisor = f"Tema: {tema}\nRoteiro Adaptado: {roteiro_adaptado}"
    # Executa o agente
    roteiro_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return roteiro_revisado

In [31]:
##########################################
# --- Agente 4: Gerador de código --- #
##########################################
def agente_gerador(tema, roteiro_revisado):
    gerador = Agent(
        name="agente_gerador",
        model="gemini-2.0-flash",
        instruction="""
            Você é um gerador de código Python que transforma histórias interativas revisadas em jogos completos de "Escolha sua Aventura", jogáveis via terminal.

            Você irá receber um roteiro revisado com múltiplos caminhos e transformar em um jogo funcional, imersivo e bem estruturado em Python.

            O jogo deve:
            Utilizar print(), input() e uma função slow_print() (com time.sleep) para imprimir o texto com clima narrativo.
            Ter múltiplas cenas com diversas escolhas por nó (2 a 4 opções).
            Utilizar uma estrutura lógica clara e modular, com variáveis como nós ou cenas, cada uma com:
            Texto da cena
            Lista de escolhas
            Transições (nó destino)
            Permitir a existência de:
            Itens adquiríveis (ex: inventário com amuletos, armas, chaves, etc.)
            Condicionais baseadas em itens (ex: "se tiver chave, abrir porta secreta")
            Finalizações diversas (bons, ruins, neutros)
            Condições lógicas simples ou múltiplas para decisões
            Permitir que o jogador jogue novamente ao final.
            Não precisa seguir exatamente uma estrutura fixa, mas o jogo deve estar completo e funcional em um único arquivo .py, legível, com boa divisão de blocos de código.

            Evite usar bibliotecas externas além de time e sys.
            """,
        description="Agente gerador de código Python"
    )
    entrada_do_agente_gerador = f"Tema: {tema}\nRoteiro Revisado: {roteiro_revisado}"
    # Executa o agente
    codigo_final = call_agent(gerador, entrada_do_agente_gerador)
    return codigo_final

In [34]:
print("🚀 Iniciando o Sistema de Criação de Jogos com 4 Agentes 🚀")

# --- Obter o Tema do Usuário ---
tema = input("❓ Por favor, digite o TEMA sobre o jogo em que você deseja criar: ")
nome_personagem_principal = input("❓ Por favor, digite o NOME do PERSONAGEM PRINCIPAL do seu jogo: ")

if not tema:
  print("Você esqueceu de digitar o tópico!")

if not nome_personagem_principal:
  print("Você esqueceu de digitar o nome do personagem principal!")
else:
  print(f"Maravilha! Vamos então criar o jogo do {nome_personagem_principal} sobre o {tema}!!")

  roteiro_inicial = agente_redator(tema, nome_personagem_principal)
  print("\n--- Resultado do Agente 1 ---\n")
  display(to_markdown(roteiro_inicial))
  print("--------------------------------------------------------------")

  roteiro_adaptado = agente_conversor(tema, roteiro_inicial)
  print("\n--- Resultado do Agente 2 ---\n")
  display(to_markdown(roteiro_adaptado))
  print("--------------------------------------------------------------")

  roteiro_revisado = agente_revisor(tema, roteiro_adaptado)
  print("\n--- Resultado do Agente 3 ---\n")
  display(to_markdown(roteiro_revisado))
  print("--------------------------------------------------------------")

  código_final = agente_gerador(tema, roteiro_revisado)
  print("\n--- Resultado do Agente 4 ---\n")
  display(to_markdown(código_final))


🚀 Iniciando o Sistema de Criação de Jogos com 4 Agentes 🚀
❓ Por favor, digite o TEMA sobre o jogo em que você deseja criar: a
❓ Por favor, digite o NOME do PERSONAGEM PRINCIPAL do seu jogo: 
Você esqueceu de digitar o nome do personagem principal!
